In [1]:
!which python

/c/Users/junec/AppData/Local/Programs/Python/Python38/python


In [2]:
import struct 
struct.calcsize("P")*8

32

In [3]:
# !pip list

In [4]:
# !pip install openpyxl

In [5]:
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
from matplotlib import rcParams
from operator import itemgetter
from typing import Callable, Dict, List, Set, Tuple
from scipy.signal import savgol_filter

In [6]:
from util import load_data_from_txt, calculate_derivative,convert_timestamp2hour,produce_pressure_4metrics
from baseline import detect_breakpoints_startPoint, detect_breakpoints, detect_breakpoints_2, detect_breakpoints_3,detect_breakpoints_4,detect_breakpoints_startPoint2
from plot import PlotNSave
from data_load_N_preprocess import LoadNPreprocessData
from plot2 import group_index, plot_breakpoints,plot_4_metrics,plot_4_metrics_details,plot_detection_statistics,plot_histogram

In [7]:
#experiment setup
# ground_truth=[2249,  2467,  2805,[2875, 2876],3100,3561,3994,[4228, 4232],4753,4895, 5297,5436,[6543, 6547], 7249,[7544, 7545],7946,[8362, 8364],9025,[9607, 9608],10149,10216, 10528,[10801, 10804],11340,  11479,12363,[13039, 13040],13868, 14086,14438,14774,15341,16098,16616,[17589, 17590],18435,19643,19764,20021,20349,20701,21030,21974,22186,22710,23220,24215,24685,25717,26006,26656, 26759,27023,[27316,27327],27773,[27966,27970],28322,28497,28832,29115]
ground_truth=[]
colum_names ={"pressure":["Elapsed time","Data","first_order_derivative","second_order_derivative"],
                    "rate":["Elapsed time","Liquid rate"]}
pressure_file="../data_input/EAGE_Shchipanov_2017/Pressure.txt"
rate_file="../data_input/EAGE_Shchipanov_2017/Rate.txt"

colum_names   ={"pressure":{"time":"Elapsed time",
                             "measure":"Data",
                             "first_order_derivative":"first_order_derivative",
                             "second_order_derivative":"second_order_derivative"},
                "rate":{"time":"Elapsed time",
                        "measure":"Liquid rate"}}
data_inOneRow=1000
pressure_filePath="../data_input/EAGE_Shchipanov_2017/Pressure.txt"
rate_filePath="../data_input/EAGE_Shchipanov_2017/Rate.txt"

In [8]:
pressure_df,rate_df,pressure_rate=load_data_from_txt(pressure_file, rate_file)
display(pressure_df)
display(rate_df)

,Elapsed time,Data
0,0.000000,2988.99
1,0.495833,2991.28
2,0.516667,2994.32
3,0.545833,2996.70
4,0.566667,2999.12
...,...,...
29808,4063.495571,7938.65
29809,4065.499737,7939.53
29810,4067.503904,7940.92
29811,4069.508071,7941.22


,Elapsed time,Liquid rate
0,0.433384,-3916.602731
1,2.516718,-3913.489043
2,4.600051,-3911.804387
3,6.641718,-3910.573563
4,7.683384,-3907.017652
...,...,...
10186,4074.862089,-18459.075655
10187,4075.028756,-18423.899451
10188,4075.237089,-18412.745895
10189,4075.362089,-18413.519153


In [9]:
# left=pressure_df.iloc[0:5]
# right=rate_df.iloc[0:5]
# result = pd.merge(left, right, how="outer", on=["Elapsed time"])
# display(left)
# display(right)
# display(result)
# b=result.sort_values(by=['Elapsed time'])
# result
# type(pressure_df.iloc[0]['Elapsed time'])
# pressure_df.iloc[1]['Elapsed time']>rate_df.iloc[0]['Elapsed time']
# b

In [10]:
#calculate first & second derivative
colum_names ={"pressure":["Elapsed time","Data","first_order_derivative","second_order_derivative"],
                    "rate":["Elapsed time","Liquid rate"]}
pressure_df=produce_pressure_4metrics(pressure_df,colum_names)
display(pressure_df)
#combine with flow rate
pressure_rate_df = pd.concat([pressure_df, rate_df])
pressure_rate_df=pressure_rate_df.sort_values(by=['Elapsed time'])
# print(pressure_df.shape[0],rate_df.shape[0],pressure_rate_df.shape[0])
# pd.set_option('display.max_rows', pressure_rate_df.shape[0]+1)
display(pressure_rate_df)

,Elapsed time,Data,first_order_derivative,second_order_derivative
0,0.000000,2988.99,4.618487,284.977831
1,0.495833,2991.28,145.919995,-3087.359588
2,0.516667,2994.32,81.600002,1184.914121
3,0.545833,2996.70,116.159996,-1817.279726
4,0.566667,2999.12,78.300001,-168.428622
...,...,...,...,...
29808,4063.495571,7938.65,0.439085,0.126970
29809,4065.499737,7939.53,0.693555,-0.271368
29810,4067.503904,7940.92,0.149688,-0.123142
29811,4069.508071,7941.22,-0.097110,0.000000


,Elapsed time,Data,first_order_derivative,second_order_derivative,Liquid rate
0,0.000000,2988.99,4.618487,284.977831,NaN
0,0.433384,NaN,NaN,NaN,-3916.602731
1,0.495833,2991.28,145.919995,-3087.359588,NaN
2,0.516667,2994.32,81.600002,1184.914121,NaN
3,0.545833,2996.70,116.159996,-1817.279726,NaN
...,...,...,...,...,...
10186,4074.862089,NaN,NaN,NaN,-18459.075655
10187,4075.028756,NaN,NaN,NaN,-18423.899451
10188,4075.237089,NaN,NaN,NaN,-18412.745895
10189,4075.362089,NaN,NaN,NaN,-18413.519153


In [11]:
colum_names   ={"pressure":{"time":"Elapsed time",
                             "measure":"Data",
                             "first_order_derivative":"first_order_derivative",
                             "second_order_derivative":"second_order_derivative"},
                "rate":{"time":"Elapsed time",
                        "measure":"Liquid rate"}}
processed_data=LoadNPreprocessData(pressure_filePath=pressure_filePath, 
                 rate_filePath=rate_filePath, 
                 colum_names=colum_names, 
                 use_SG_smoothing=False)
pressure_df=processed_data.pressure_df
rate_df=processed_data.rate_df

display(pressure_df)
display(rate_df)
display(processed_data.pressureNrate_df)

first_order_derivative=pressure_df["first_order_derivative"]
second_order_derivative=pressure_df["second_order_derivative"]

---load data from 'txt' or 'xlsx' files...
check the time type
---The first & second order derivative has been calculated and appended to pressure dataframe


,Elapsed time,Data,first_order_derivative,second_order_derivative
0,0.000000,2988.99,4.618487,284.977831
1,0.495833,2991.28,145.919995,-3087.359588
2,0.516667,2994.32,81.600002,1184.914121
3,0.545833,2996.70,116.159996,-1817.279726
4,0.566667,2999.12,78.300001,-168.428622
...,...,...,...,...
29808,4063.495571,7938.65,0.439085,0.126970
29809,4065.499737,7939.53,0.693555,-0.271368
29810,4067.503904,7940.92,0.149688,-0.123142
29811,4069.508071,7941.22,-0.097110,0.000000


,Elapsed time,Liquid rate
0,0.433384,-3916.602731
1,2.516718,-3913.489043
2,4.600051,-3911.804387
3,6.641718,-3910.573563
4,7.683384,-3907.017652
...,...,...
10186,4074.862089,-18459.075655
10187,4075.028756,-18423.899451
10188,4075.237089,-18412.745895
10189,4075.362089,-18413.519153


,Elapsed time,Data,first_order_derivative,second_order_derivative,Liquid rate
0,0.000000,2988.99,4.618487,284.977831,NaN
0,0.433384,NaN,NaN,NaN,-3916.602731
1,0.495833,2991.28,145.919995,-3087.359588,NaN
2,0.516667,2994.32,81.600002,1184.914121,NaN
3,0.545833,2996.70,116.159996,-1817.279726,NaN
...,...,...,...,...,...
10186,4074.862089,NaN,NaN,NaN,-18459.075655
10187,4075.028756,NaN,NaN,NaN,-18423.899451
10188,4075.237089,NaN,NaN,NaN,-18412.745895
10189,4075.362089,NaN,NaN,NaN,-18413.519153


In [12]:
# #plot histogram
# data=pressure_df["first_order_derivative"]
# xlabel="first_order_derivative"
# ylabel="data count"
# title='histogram--real field data'
# plot_histogram(data, xlabel, ylabel,title)

In [13]:
# breakpoints_detected=detect_breakpoints(first_order_derivative)
# print(len(breakpoints_detected))
# print(breakpoints_detected)

In [14]:
# breakpoints_detected=detect_breakpoints_2(first_order_derivative,second_order_derivative)
# print(len(breakpoints_detected))
# # print(breakpoints_detected)
# plot_detection_statistics(breakpoints_detected,ground_truth)
# plot_4_metrics(pressure_df,
#                rate_df,
#                breakpoints_detected,
#                ground_truth,
#                colum_names)

**without denosing**

In [15]:
#baseline method
first_order_derivative=pressure_df["first_order_derivative"]
second_order_derivative=pressure_df["second_order_derivative"]
noise_threshold=2
breakpoints_detected_3=detect_breakpoints_3(first_order_derivative,second_order_derivative,noise_threshold)
print(len(breakpoints_detected_3))
# print(breakpoints_detected)
plot_detection_statistics(breakpoints_detected_3,ground_truth)

993.2336512455328 192914.8008199997
116
No ground truth defined


In [16]:
plot_4_metrics(pressure_df,
               rate_df,
               breakpoints_detected_3,
               ground_truth,
               colum_names)

TypeError: plot_4_metrics() missing 1 required positional argument: 'plot_name'

In [ ]:
#baseline method
data_inOneRow=700
plot_4_metrics_details(data_inOneRow,
                       pressure_df,
                       rate_df,
                       breakpoints_detected_3,
                       ground_truth,
                       colum_names)

**S-G filter**

In [ ]:
#denosing using S-G filter
pressure_df,rate_df,pressure_rate=load_data_from_txt(pressure_file, rate_file)
denoised_pressure_data=savgol_filter(pressure_df["Data"],199,3)

data={"Elapsed time":pressure_df["Elapsed time"],"Data":denoised_pressure_data}
pressure_df_denoised=pd.DataFrame(data=data)
pressure_df_denoised=produce_pressure_4metrics(pressure_df_denoised,colum_names)
# display(pressure_df_denoised)
first_order_derivative=pressure_df_denoised["first_order_derivative"]
second_order_derivative=pressure_df_denoised["second_order_derivative"]
std_1=statistics.stdev(first_order_derivative)
std_2=statistics.stdev(second_order_derivative)
print(f"std_1:{std_1},std_2:{std_2}")

#plot first 700 denoised data
pressure_df_first400=pressure_df.iloc[0:400,:]
fig=plt.figure(figsize=(9,4))
ax=fig.subplots()
p=ax.plot(pressure_df_first400["Elapsed time"],pressure_df_first400["Data"],label='raw data')
p=ax.plot(pressure_df_first400["Elapsed time"],denoised_pressure_data[0:400],label='S-G filtered')
legend = ax.legend(loc='upper left', shadow=True, fontsize='x-large')
# legend.get_frame().set_facecolor('C0')
plt.show()

In [ ]:
# #S-G filter startPoint
# noise_threshold=3
# breakpoints_detected_startPoint=detect_breakpoints_startPoint(first_order_derivative,noise_threshold)
# print("len(breakpoints_detected)",len(breakpoints_detected_startPoint))
# plot_4_metrics(pressure_df_denoised,
#                rate_df,
#                breakpoints_detected_startPoint,
#                ground_truth,
#                colum_names)

In [ ]:
# data_inOneRow=800
# plot_4_metrics_details(data_inOneRow,
#                        pressure_df_denoised,
#                        rate_df,
#                        breakpoints_detected_startPoint,
#                        ground_truth,
#                        colum_names)

In [ ]:
#S-G filter startPoint2
noise_threshold=3
breakpoints_detected_startPoint2=detect_breakpoints_startPoint2(first_order_derivative,second_order_derivative,noise_threshold)
print("len(breakpoints_detected)",len(breakpoints_detected_startPoint2))
plot_4_metrics(pressure_df_denoised,
               rate_df,
               breakpoints_detected_startPoint2,
               ground_truth,
               colum_names)

In [ ]:
#S-G filter detect algorithm 3
noise_threshold=0.3
breakpoints_detected_3SG=detect_breakpoints_3(first_order_derivative,second_order_derivative,noise_threshold)
print(len(breakpoints_detected_3SG))
plot_4_metrics(pressure_df_denoised,
               rate_df,
               breakpoints_detected_3SG,
               ground_truth,
               colum_names)

In [ ]:
data_inOneRow=700
plot_4_metrics_details(data_inOneRow,
                       pressure_df_denoised,
                       rate_df,
                       breakpoints_detected_3SG,
                       ground_truth,
                       colum_names)

In [ ]:
#S-G filter detect algorithm 4
# noise_threshold=0.3
# close_zero_threshold=0.5
# window_time_duration=8
# breakpoints_detected_4SG=detect_breakpoints_4(pressure_df_denoised,colum_names, noise_threshold,window_time_duration,close_zero_threshold)
# print(len(breakpoints_detected_4SG))
# plot_4_metrics(pressure_df_denoised,
#                rate_df,
#                breakpoints_detected_4SG,
#                ground_truth,
#                colum_names)

In [ ]:
# notin3=[point for point in breakpoints_detected_4SG if point not in breakpoints_detected_3SG]
# print(len(breakpoints_detected_3SG),len(breakpoints_detected_4SG),len(notin3))
# data_inOneRow=700
# plot_4_metrics_details(data_inOneRow,
#                        pressure_df_denoised,
#                        rate_df,
#                        breakpoints_detected_3SG,
#                        breakpoints_detected_4SG,
#                        colum_names)

In [ ]:
data_inOneRow=700
# plot_4_metrics_details(data_inOneRow,
#                        pressure_df_denoised,
#                        rate_df,
#                        breakpoints_detected_4SG,
#                        ground_truth,
#                        colum_names)

In [ ]:
len(breakpoints_detected_3),len(breakpoints_detected_3SG)
index_zero_rate=[index for index, rate in enumerate(rate_df["Liquid rate"]) if rate==0]
print(index_zero_rate)
indexo_rate=[rate_df.iloc[index]["Elapsed time"] for index, rate in enumerate(rate_df["Liquid rate"]) if rate==0]
time_zero_rate=[rate_df.iloc[index]["Elapsed time"] for index, rate in enumerate(rate_df["Liquid rate"]) if rate==0]
len(time_zero_rate),time_zero_rate

In [ ]:
breakpoints_zero_rate=[]
for time_rate in time_zero_rate:
    for index, time_pressure in enumerate(pressure_df["Elapsed time"]):
        if time_pressure>=time_rate:
            breakpoints_zero_rate. append(index)
            break
len(breakpoints_zero_rate) 
breakpoints_zero_rate
zero_rate={"index_zero_rate":index_zero_rate,"breakpoints_zero_rate":breakpoints_zero_rate}
zero_rate_df=pd.DataFrame(data=zero_rate)
zero_rate_df

In [ ]:
rate_df.iloc[1489]["Elapsed time"]
for index, time_pressure in enumerate(pressure_df["Elapsed time"]):
        if time_pressure>=rate_df.iloc[2893]["Elapsed time"]:
            print(index)
            break

In [ ]:
print(len(breakpoints_detected_3+breakpoints_detected_3SG+breakpoints_zero_rate+[5297]))
breakpoints_union=set(breakpoints_detected_3+breakpoints_detected_3SG+breakpoints_zero_rate+[5297]+[10216]+[10528]+[14774]+[16616]+[19764]+[20021]+[21030]+[25717]+[28322])
print(len(breakpoints_union))
# breakpoints_union

In [ ]:
rate_df.iloc[1490], rate_df.iloc[1489]["Elapsed time"]

In [ ]:
a=pressure_df.iloc[10150:10245]["Data"]
min(a)
pressure_df.loc[pressure_df["Data"]==min(a)]
pd.set_option('display.max_rows', pressure_df.shape[0]+1)
display(pressure_df.iloc[10150:10245])
delta=pressure_df.iloc[10151]["Elapsed time"]-pressure_df.iloc[10150]["Elapsed time"]
for i in range(10150,10223):
    t=pressure_df.iloc[i+1]["Elapsed time"]-pressure_df.iloc[i]["Elapsed time"]
    print(i,t,delta)
    if t>delta:
        delta=t
        pointer=i
i

In [ ]:
# data_inOneRow=700
# plot_4_metrics_details(data_inOneRow,
#                        pressure_df,
#                        rate_df,
#                        breakpoints_zero_rate,
#                        ground_truth,
#                        colum_names)

In [ ]:
# data_inOneRow=500
# plot_4_metrics_details(data_inOneRow,
#                        pressure_df,
#                        rate_df,
#                        list(breakpoints_union),
#                        ground_truth,
#                        colum_names)

In [ ]:
bp=28310
pressure_df.iloc[bp-30:bp+30]

In [ ]:
a=[10245, 10499, 10021, 10149, 10150]
b=a.sort()
a,b

In [ ]:
breakpoints_union=set(breakpoints_detected_3SG+breakpoints_zero_rate+[5297]+[10216]+[10528]+[11340]+[12363]+[13868]+[14774]+[16616]+[16098]+[19764]+[20021]+[21030]+[25717]+[28322])
print(len(breakpoints_union))
# breakpoints_union

In [ ]:
data_inOneRow=500
# plot_4_metrics_details(data_inOneRow,
#                        pressure_df,
#                        rate_df,
#                        list(breakpoints_union),
#                        ground_truth,
#                        colum_names)